# **Homework**: Data talks club data engineering zoomcamp Data loading workshop

Hello folks, let's practice what we learned - Loading data with the best practices of data engineering.

Here are the exercises we will do




# 1. Use a generator

Remember the concept of generator? Let's practice using them to futher our understanding of how they work.

Let's define a generator and then run it as practice.

**Answer the following questions:**

- **Question 1: What is the sum of the outputs of the generator for limit = 5?**
- **Question 2: What is the 13th number yielded**

I suggest practicing these questions without GPT as the purpose is to further your learning.

In [1]:
def square_root_generator(limit):
    n = 1
    while n <= limit:
        yield n ** 0.5
        n += 1

# Example usage:
limit = 5
generator = square_root_generator(limit)

for sqrt_value in generator:
    print(sqrt_value)


1.0
1.4142135623730951
1.7320508075688772
2.0
2.23606797749979


**Question 1: What is the sum of the outputs of the generator for limit = 5?**

In [2]:
limit = 5
generator = square_root_generator(limit)

sum_of_outputs = 0

for output in generator:
  sum_of_outputs += output

print(sum_of_outputs)

8.382332347441762


**Question 2: What is the 13th number yielded**

In [3]:
limit = 13
generator = square_root_generator(limit)

for output in generator:
    pass

print(output)

3.605551275463989


# 2. Append a generator to a table with existing data


Below you have 2 generators. You will be tasked to load them to duckdb and answer some questions from the data

1. Load the first generator and calculate the sum of ages of all people. Make sure to only load it once.
2. Append the second generator to the same table as the first.
3. **After correctly appending the data, calculate the sum of all ages of people.**




In [4]:
#Install the dependencies
%%capture
!pip install dlt[duckdb]

In [5]:
import dlt

# first generator
def people_1():
    for i in range(1, 6):
        yield {"ID": i, "Name": f"Person_{i}", "Age": 25 + i, "City": "City_A"}

# second generator
def people_2():
    for i in range(3, 9):
        yield {"ID": i, "Name": f"Person_{i}", "Age": 30 + i, "City": "City_B", "Occupation": f"Job_{i}"}

# define the connection to load to.
generators_pipeline = dlt.pipeline(destination='duckdb', dataset_name='generators')


# load the first generator to a table:
info = generators_pipeline.run(people_1(),
										table_name="appended_table",
										write_disposition="replace")

# inspect the load outcome of the first generator
print(info)

# append the second generator to the same table:
info = generators_pipeline.run(people_2(),
										table_name="appended_table",
										write_disposition="append")

# inspect the load outcome of the second generator
print(info)

Pipeline dlt_colab_kernel_launcher load step completed in 0.40 seconds
1 load package(s) were loaded to destination duckdb and into dataset generators
The duckdb destination used duckdb:////content/dlt_colab_kernel_launcher.duckdb location to store data
Load package 1708270220.6966932 is LOADED and contains no failed jobs
Pipeline dlt_colab_kernel_launcher load step completed in 0.40 seconds
1 load package(s) were loaded to destination duckdb and into dataset generators
The duckdb destination used duckdb:////content/dlt_colab_kernel_launcher.duckdb location to store data
Load package 1708270221.5443091 is LOADED and contains no failed jobs


In [6]:
import duckdb

# set up a connection to duckdb
conn = duckdb.connect(f"{generators_pipeline.pipeline_name}.duckdb")

# display the appended table
result = conn.sql("SELECT * FROM generators.appended_table").df()
display(result)

# display the sum of age of all people
age_total = conn.sql("SELECT SUM(Age) FROM generators.appended_table").fetchall()
print()
print(f"Sum of age of all people: {age_total[0][0]}")

,id,name,age,city,_dlt_load_id,_dlt_id,occupation
0,1,Person_1,26,City_A,1708270220.6966932,zE6jLqKy4mi7kw,None
1,2,Person_2,27,City_A,1708270220.6966932,Ajti4L0vmk2oSw,None
2,3,Person_3,28,City_A,1708270220.6966932,G5Lis5cm93Af0g,None
3,4,Person_4,29,City_A,1708270220.6966932,x033tJhGZvVMHw,None
4,5,Person_5,30,City_A,1708270220.6966932,5mfeleGPl7qmBA,None
5,3,Person_3,33,City_B,1708270221.5443091,w9eYH9WM1fz+2Q,Job_3
6,4,Person_4,34,City_B,1708270221.5443091,Au4Abzqyjj+/2A,Job_4
7,5,Person_5,35,City_B,1708270221.5443091,LT1Fs1R7Wew4Sg,Job_5
8,6,Person_6,36,City_B,1708270221.5443091,Hqixe/Lkk5IAUw,Job_6
9,7,Person_7,37,City_B,1708270221.5443091,SWFXqKT8Sju3Xw,Job_7



Sum of age of all people: 353


# 3. Merge a generator

Re-use the generators from Exercise 2.

A table's primary key needs to be created from the start, so load your data to a new table with primary key ID.

Load your first generator first, and then load the second one with merge. Since they have overlapping IDs, some of the records from the first load should be replaced by the ones from the second load.

After loading, you should have a total of 8 records, and ID 3 should have age 33.

Question: **Calculate the sum of ages of all the people loaded as described above.**


In [7]:
# define the connection to load to.
generators_pipeline = dlt.pipeline(destination='duckdb', dataset_name='generators')


# load the first generator to a table:
info = generators_pipeline.run(people_1(),
                               table_name="merged_table",
                               write_disposition="merge",
                               primary_key="ID")

# inspect the load outcome of the first generator
print(info)

# merge the second generator to the same table:
info = generators_pipeline.run(people_2(),
                               table_name="merged_table",
                               write_disposition="merge",
                               primary_key="ID")

# inspect the load outcome of the second generator
print(info)

Pipeline dlt_colab_kernel_launcher load step completed in 0.65 seconds
1 load package(s) were loaded to destination duckdb and into dataset generators
The duckdb destination used duckdb:////content/dlt_colab_kernel_launcher.duckdb location to store data
Load package 1708270222.8659246 is LOADED and contains no failed jobs
Pipeline dlt_colab_kernel_launcher load step completed in 0.39 seconds
1 load package(s) were loaded to destination duckdb and into dataset generators
The duckdb destination used duckdb:////content/dlt_colab_kernel_launcher.duckdb location to store data
Load package 1708270224.0230045 is LOADED and contains no failed jobs


In [8]:
# set up a connection to duckdb
conn = duckdb.connect(f"{generators_pipeline.pipeline_name}.duckdb")

# display the appended table
result = conn.sql("SELECT * FROM generators.merged_table").df()
display(result)

# display the sum of age of all people
age_total = conn.sql("SELECT SUM(Age) FROM generators.merged_table").fetchall()
print()
print(f"Sum of age of all people: {age_total[0][0]}")

,id,name,age,city,_dlt_load_id,_dlt_id,occupation
0,2,Person_2,27,City_A,1708270222.8659246,gbSW3qmQQYwHtQ,None
1,1,Person_1,26,City_A,1708270222.8659246,MkteXph6Y0vGCQ,None
2,5,Person_5,35,City_B,1708270224.0230045,NJXvvZqM64/qyw,Job_5
3,7,Person_7,37,City_B,1708270224.0230045,WLtLeOCxdNdxmA,Job_7
4,4,Person_4,34,City_B,1708270224.0230045,UNu7zfKM4xf08A,Job_4
5,8,Person_8,38,City_B,1708270224.0230045,/u5np8APfhNFIQ,Job_8
6,3,Person_3,33,City_B,1708270224.0230045,sMnEl1oq87I0Dw,Job_3
7,6,Person_6,36,City_B,1708270224.0230045,OkIl9VfCPguwkw,Job_6



Sum of age of all people: 266


Questions? difficulties? We are here to help.
- DTC data engineering course channel: https://datatalks-club.slack.com/archives/C01FABYF2RG
- dlt's DTC cohort channel: https://dlthub-community.slack.com/archives/C06GAEX2VNX